In [1]:
!pip install bs4
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'html.parser')
print(soup.div)

<div class="noprint" id="mw-page-base"></div>


In [3]:
# getting the table with bs4
table = soup.find('table')
columns = table.find_all('th')
rows = [t for t in table.find_all('tr') if not t.find('th')]

### Preprocess rows
Let's remove cells that have a borough as 'Not Assigned' and change format to string for easier analysis

In [4]:
td_rows = [[x for x in t.contents if x!='\n'] for t in rows]

In [5]:
filtered_td_rows = [x for x in td_rows if x[1].text!='Not assigned']
print("length before filter : {}\nlength after filter : {}".format(len(td_rows),len(filtered_td_rows)))

length before filter : 288
length after filter : 211


In [6]:
for idx,row in enumerate(filtered_td_rows):
    # let's get the text only first
    for i in range(len(row)):
        filtered_td_rows[idx][i] = row[i].text
        if('\n' in filtered_td_rows[idx][i]):
            filtered_td_rows[idx][i] = filtered_td_rows[idx][i][:-1]
    # change neighborhood if it is not assigned with borough
    if(row[2]=='Not assigned'):
        filtered_td_rows[idx][2] = filtered_td_rows[idx][1]

In [7]:
print(filtered_td_rows[:5])

[['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront'], ['M5A', 'Downtown Toronto', 'Regent Park'], ['M6A', 'North York', 'Lawrence Heights']]


Now let's fuse duplicate postal code

In [8]:
previousPC = ''
final_rows,fused_list = [],[]
for i in range(len(filtered_td_rows)):
    # pass rows that are already fused
    if(filtered_td_rows[i][0] in fused_list): continue
    # check if it is the same postal code than previous one
    if(previousPC==filtered_td_rows[i][0]):
        tmp = [final_rows[-1][0],final_rows[-1][1],final_rows[-1][2]]
        outOfBound = False
        j = i 
        while(previousPC==filtered_td_rows[j][0]):
            tmp[2] += ", "+filtered_td_rows[j][2]
            j+=1
            if(j>=len(filtered_td_rows)): 
                outOfBound = True
                break
        final_rows.pop()
        final_rows.append(tmp)  
        fused_list.append(final_rows[-1][0])
    else:
        final_rows.append(filtered_td_rows[i])
        previousPC = filtered_td_rows[i][0]
print(final_rows[:10])

[['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront, Regent Park'], ['M6A', 'North York', 'Lawrence Heights, Lawrence Manor'], ['M7A', "Queen's Park", "Queen's Park"], ['M9A', 'Etobicoke', 'Islington Avenue'], ['M1B', 'Scarborough', 'Rouge, Malvern'], ['M3B', 'North York', 'Don Mills North'], ['M4B', 'East York', 'Woodbine Gardens, Parkview Hill'], ['M5B', 'Downtown Toronto', 'Ryerson, Garden District']]


First let's get the strings for our columns

In [9]:
columns_text = []
for c in columns:
    if('\n' not in c.text):
        columns_text.append(c.text)
    else:
        columns_text.append(c.text[:-1])

In [10]:
columns_text

['Postcode', 'Borough', 'Neighbourhood']

Now let's create the dataframe

In [11]:
pc_rows = [x[0] for x in final_rows]
borough_rows = [x[1] for x in final_rows]
neighborhood_rows = [x[2] for x in final_rows]

In [12]:
df = pd.DataFrame({columns_text[0]:pc_rows,columns_text[1]:borough_rows,columns_text[2]:neighborhood_rows})
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [13]:
df.rename(index=str,columns={'Postcode':'PostalCode','Neighbourhood':'Neighborhood'},inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [16]:
df.to_csv('neigh.csv',index=False)

In [14]:
df.shape

(103, 3)